# 05_test_model_by_unseen_data.ipynb


In [7]:
import pandas as pd
import numpy as np
import joblib
from geopy.distance import geodesic

# Load trained pipeline model


In [8]:
model = joblib.load('../models/best_model.pkl')

# Define mappings (must match training-time encodings)


In [9]:
type_of_order_map = {'Buffet': 0, 'Drinks': 1, 'Meal': 2, 'Snack': 3}
type_of_vehicle_map = {'bicycle': 0, 'electric_scooter': 1, 'motorcycle': 2, 'scooter': 3}

def prepare_input(raw_dict):
    df = pd.DataFrame([raw_dict])

    # Calculate distance
    origin = (raw_dict['Restaurant_latitude'], raw_dict['Restaurant_longitude'])
    destination = (raw_dict['Delivery_location_latitude'], raw_dict['Delivery_location_longitude'])
    distance_km = geodesic(origin, destination).km
    df['distance_km'] = distance_km

    # Feature engineering
    df['log_distance'] = np.log(distance_km + 1e-5)
    df['distance_x_rating'] = distance_km * raw_dict['Delivery_person_Ratings']
    df['age_x_rating'] = raw_dict['Delivery_person_Age'] * raw_dict['Delivery_person_Ratings']
    df['speed_estimate'] = distance_km / 0.5  # simple rule

    # Encode categorical features
    df['Type_of_order_encoded'] = type_of_order_map[raw_dict['Type_of_order']]
    df['Type_of_vehicle_encoded'] = type_of_vehicle_map[raw_dict['Type_of_vehicle']]

    # Age group features
    df['Age_group_25-34'] = 1 if 25 <= raw_dict['Delivery_person_Age'] < 35 else 0
    df['Age_group_35-44'] = 1 if 35 <= raw_dict['Delivery_person_Age'] < 45 else 0
    df['Age_group_45-60'] = 1 if 45 <= raw_dict['Delivery_person_Age'] <= 60 else 0

    # Drop unused raw columns
    return df[model.feature_names_in_]

# Example input for prediction


In [10]:
raw_input = {
    'Delivery_person_Age': 30,
    'Delivery_person_Ratings': 4.7,
    'Restaurant_latitude': 12.9716,
    'Restaurant_longitude': 77.5946,
    'Delivery_location_latitude': 12.9352,
    'Delivery_location_longitude': 77.6141,
    'Type_of_order': 'Meal',
    'Type_of_vehicle': 'motorcycle'
}

# Prepare and predict


In [11]:
input_prepared = prepare_input(raw_input)
predicted_time = model.predict(input_prepared)

print(f"\nInput Summary: {raw_input}")
print(f"Predicted Delivery Time: {predicted_time[0]:.2f} minutes")


Input Summary: {'Delivery_person_Age': 30, 'Delivery_person_Ratings': 4.7, 'Restaurant_latitude': 12.9716, 'Restaurant_longitude': 77.5946, 'Delivery_location_latitude': 12.9352, 'Delivery_location_longitude': 77.6141, 'Type_of_order': 'Meal', 'Type_of_vehicle': 'motorcycle'}
Predicted Delivery Time: 5.37 minutes
